In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.applications.vgg16 import VGG16

# Define data generators
batch_size = 32
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# Load data
dataset_path = '/content/drive/MyDrive/PBL-3/train'
classes = ['Acne', 'Actinic Keratosis', 'Basal Cell Carcinoma', 'Eczemaa', 'Rosacea']
train_generator = train_datagen.flow_from_directory(
        dataset_path,
        classes=classes,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')


Found 635 images belonging to 5 classes.


In [ ]:
# Load VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create new model
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Freeze VGG16 layers
for layer in vgg16_model.layers:
    layer.trainable = False

In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=10)

<ipython-input-20-0f68a8e0fab9>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
19/19 [==============================] - 18s 883ms/step - loss: 3.0977 - accuracy: 0.2968
Epoch 2/10
19/19 [==============================] - 12s 620ms/step - loss: 1.3775 - accuracy: 0.4212
Epoch 3/10
19/19 [==============================] - 12s 612ms/step - loss: 1.2429 - accuracy: 0.4925
Epoch 4/10
19/19 [==============================] - 12s 622ms/step - loss: 1.2246 - accuracy: 0.4809
Epoch 5/10
19/19 [==============================] - 12s 617ms/step - loss: 1.2613 - accuracy: 0.4760
Epoch 6/10
19/19 [==============================] - 13s 655ms/step - loss: 1.1747 - accuracy: 0.5124
Epoch 7/10
19/19 [==============================] - 12s 622ms/step - loss: 1.0836 - accuracy: 0.5638
Epoch 8/10
19/19 [==============================] - 12s 625ms/step - loss: 1.0283 - accuracy: 0.5837
Epoch 9/10
19/19 [==============================] - 12s 626ms/step - loss: 1.0258 - accuracy: 0.5738
Epoch 10/10
19/19 [==============================] - 12s 613ms/step - loss: 1.0002 - accura

In [ ]:
# Save the model as an .h5 file
model.save('model_weights_VGG16.h5')